## MPF Evaluation Framework: ImageNet Validation Script w/ FiftyOne

This script provides an example of the FiftyOne tool for processing ImageNet Results.
Please feel free to use and modify the script as needed for investigations.

In [1]:
#############################################################################
# NOTICE                                                                    #
#                                                                           #
# This software (or technical data) was produced for the U.S. Government    #
# under contract, and is subject to the Rights in Data-General Clause       #
# 52.227-14, Alt. IV (DEC 2007).                                            #
#                                                                           #
# Copyright 2021 The MITRE Corporation. All Rights Reserved.                #
#############################################################################

#############################################################################
# Copyright 2021 The MITRE Corporation                                      #
#                                                                           #
# Licensed under the Apache License, Version 2.0 (the "License");           #
# you may not use this file except in compliance with the License.          #
# You may obtain a copy of the License at                                   #
#                                                                           #
#    http://www.apache.org/licenses/LICENSE-2.0                             #
#                                                                           #
# Unless required by applicable law or agreed to in writing, software       #
# distributed under the License is distributed on an "AS IS" BASIS,         #
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  #
# See the License for the specific language governing permissions and       #
# limitations under the License.                                            #
#############################################################################

### Import Dependencies

In [2]:
import argparse
import fiftyone as fo
import csv
import numpy as np
import json
import os
import sys
import random
import datetime
import pandas as pd

from pathlib import Path

### Specify ImageNet Dataset, MPF JSON Predictions Directory, and Related Annotation Files:

In [3]:
# Path to ImageNet Dataset Directory
dataset_dir = "/home/mpf/G_Drive/ImageNet/val"

# Path to dataset annotations file or directory (csv for ImageNet).
annotations_file = "/home/mpf/G_Drive/ImageNet/ClipMetrics/data/LOC_val_solution.csv"

# Path to class labels file (classes.txt for ImageNet).
class_mapping_file = "/home/mpf/G_Drive/ImageNet/ClipMetrics/data/classifications.txt"

# Path to predictions directory containing job run subdirectories.
predictions_dir = "/home/mpf/G_Drive/ImageNet/ClipMetrics/outputs/"

# Output path for prediction metrics.
output_metrics = "./results.txt"


In [4]:
# Specifies number of files to process in dataset.
# Set below 0 to process the entire dataset, or between 0-1 to process a percentage of the dataset.
dataset_subsample = -1

Below are optional parameters:

In [5]:
# Dataset type (Currently supports "imagenet")
dataset_type = "imagenet"

#Enable FiftyOne Interface:
view_fiftyone = True
view_fiftyone_port = 5151

# Enable reporting of all individual class metrics.
# Warning, label file can be hard to read.
verbose = False

# Checks if class labels should be standardized to lower case."
case_sensitive = True

# Suppresses print warnings.
suppress_warnings = True

## MPF Evaluation Framework Class:

In [6]:
class EvalFramework:
    def __enter__(self):
        return self

    def __exit__(self, *exc_info):
        return

    def __init__(self,
                 dataset_dir: str = None,
                 annotations_file: str = None,
                 predictions_dir: str = None,
                 dataset_type: str = None,
                 output_metrics: str = None,
                 view_fiftyone: bool = True,
                 class_mapping_file: str = None,
                 case_sensitive: bool = True,
                 dataset_sample: float = -1,
                 verbose: bool = False,
                 suppress_warnings = True,
                 ):
        
        self.dataset_dir = dataset_dir
        self.annotations_file = annotations_file
        self.predictions_dir = predictions_dir
        # TODO: Add support for other dataset types
        self.dataset_type = dataset_type
        self.class_mapping_file = class_mapping_file

        self.view_fiftyone = view_fiftyone
        self.case_sensitive = case_sensitive
        self.output_metrics = output_metrics

        self.prediction_tags = []

        self.dataset_sample = dataset_sample

        self.class_to_string = {}
        self.class_id_list = []
        self.annotations = []
        self.dataset = fo.Dataset()
        self.results = {}
        self.verbose = verbose
        self.suppress_warnings = suppress_warnings

        if self.dataset_type == "imagenet":
            self._init_imagenet_data()

    def _init_imagenet_data(self):
        """
        Initialize the ImageNet class labels list and ImageNet ground truth annotations.
        :return:
        """

        self._setup_class_lists_imagenet()
        self._setup_imagenet_annotations()

    def _setup_class_lists_imagenet(self):
        """
        Initialize the ImageNet class labels list.
        :return:
        """

        with open(self.class_mapping_file, "r+") as f:
            for id_string in f.read().split("\n"):
                id_list = id_string.split(" ")
                self.class_to_string[id_list[0]] = id_string[len(id_list[0]):].strip()
                if not self.case_sensitive:
                    self.class_to_string[id_list[0]] = self.class_to_string[id_list[0]].lower()
                self.class_id_list.append(self.class_to_string[id_list[0]])

    def _setup_imagenet_annotations(self):
        """
        Initialize the ImageNet groundtruth dataset annotations.
        Based on code by Zachary Cafego.
        :return:
        """
        with open(self.annotations_file) as f:
            csv_reader = csv.reader(f, delimiter=",")
            line_count = 0
            for row in csv_reader:
                line_count += 1
                if line_count > 1:
                    filename = row[0]
                    classification = row[1].split()[0]
                    filepath = os.path.join(self.dataset_dir, filename+'.JPEG')
                    if not os.path.exists(filepath):
                        continue
                    self.annotations.append(dict(filename=filename, filepath=filepath, classification=classification))

    def setup_dataset(self):
        """
        Setup ground truth dataset for image classification.
        :return:
        """
        samples = []
        for annotation in self.annotations:
            filepath = annotation["filepath"]
            label = annotation["classification"]
            sample = fo.Sample(filepath=filepath)

            # Store classification in a field name of your choice
            label = self.class_to_string[label]
            if not self.case_sensitive:
                label = label.lower()

            sample["ground_truth"] = fo.Classification(label=label)
            samples.append(sample)

        dataset_samples = samples

        # Trim dataset if user enters a fraction or non-negative number of files.
        if self.dataset_sample > 0:
            if self.dataset_sample < 1:
                dataset_sample = len(samples) * self.dataset_sample
            dataset_samples = random.sample(samples, dataset_sample)
        self.dataset.add_samples(dataset_samples)

    def _create_logits(self, top_preds, class_list):
        """
        Create a logits prediction vector given a list of top predictions.

        :param top_preds:
        :param class_list:
        :return:
        """
        logits = np.full(len(class_list), sys.float_info.min)
        for pred in top_preds:
            # print(pred)
            logits[class_list.index(pred[0])] = pred[1]
        return np.log(logits)

    def add_predictions(self):
        """
        Add predictions from each job to dataset.
        :return:
        """
        # Construct a logits array and fill in top predictions by model.
        # All missing predictions are reconstructed with lowest possible values.
        predictions_list = [f.path for f in os.scandir(self.predictions_dir) if f.is_dir()]

        for pred in predictions_list:

            pred_tag = pred.split("/")[-1]
            self.prediction_tags.append(pred_tag)
            for sample in self.dataset:
                filepath = sample["filepath"]
                filename = filepath.split("/")[-1][:-5]
                jfile = os.path.join(pred, filename + ".json")
                if os.path.exists(jfile):
                    if str(filepath) in self.dataset:
                        with open(jfile) as f:
                            output = json.load(f)
                            top_classes = output["media"][0]["output"]["CLASS"][0]["tracks"][0]["detections"][0][
                                "detectionProperties"]["CLASSIFICATION LIST"]
                            top_conf = output["media"][0]["output"]["CLASS"][0]["tracks"][0]["detections"][0][
                                "detectionProperties"]["CLASSIFICATION CONFIDENCE LIST"]
                            top_conf = [float(f.strip(" tensor()")) for f in top_conf.split(";")]
                            if not self.case_sensitive:
                                top_classes = [x.lower() for x in top_classes.split("; ")]
                            else:
                                top_classes = top_classes.split("; ")

                            logits = self._create_logits(zip(top_classes, top_conf), self.class_id_list)

                            sample = self.dataset[str(filepath)]
                            sample[pred_tag] = fo.Classification(
                                label=top_classes[0],
                                confidence=top_conf[0],
                                logits=logits,
                            )
                            sample.save()
                    else:
                        if not self.suppress_warnings:
                            print("Warning: Missing Groundtruth for " + filepath)
                else:
                    if not self.suppress_warnings:
                        print("Warning: Missing Prediction for " + filepath)
                        self.dataset.remove_sample(sample)

    def evaluate_fiftyone(self):
        """
        Evaluate top k predictions for each job run.
        K values range from 1-10 and can be adjusted for further inferences.
        :return:
        """

        #TODO: Continue updating evaluation framework metrics.
        k_pred = [1, 3, 5, 10]
        for pred in self.prediction_tags:
            results_dict = {}
            for k in k_pred:
                results = self.dataset.evaluate_classifications(
                    pred,
                    gt_field="ground_truth",
                    eval_key="eval_" + pred.replace("-","_") + "_k_{}".format(k),
                    method="top-k",
                    classes=self.class_id_list,
                    k=k,
                )
                results_dict[k] = results.report(classes=self.class_id_list)

            if self.output_metrics is not None:
                self.results[pred] = results_dict

    def launch_fiftyone_session(self, port: int = 5151):
        """
        Launch a FiftyOne session if prompted by the user.
        :return: None
        """
        session = fo.launch_app(self.dataset, port=port)
        closing = input("Would you like to close this app (y/n):")
        while closing.lower()[0] != "y":
            closing = input("Would you like to close this app (y/n):")

    def generate_summary(self):
        """
        Print out summary of metric results.
        :return:
        """
        Path(self.output_metrics).parent.mkdir(parents=True, exist_ok=True)
        if len(self.results) > 0:
            with open(self.output_metrics, "w+") as f:
                f.write("MPF Evaluation Summary:\n")
                f.write("Date: {}\n".format(datetime.datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")))

                for job_name, result in self.results.items():
                    f.write("\n\nRun Name: {}\n".format(job_name))
                    for k in result:
                        f.write("Prediction Metrics (top-k = {}): \n\n".format(k))
                        if self.verbose:
                            metrics = pd.DataFrame(result[k]).transpose()
                        else:
                            aggregate_res = ["micro avg", "macro avg", "weighted avg"]
                            metrics = {}
                            for agg in aggregate_res:
                                metrics[agg] = result[k][agg]
                            metrics = pd.DataFrame(metrics).transpose()
                        f.write(metrics.to_csv(sep="\t"))
                        f.write("\n")
                        
        return self.results

#### Initialize evalutor

In [7]:
evaluator = EvalFramework(dataset_dir=dataset_dir,
                       annotations_file=annotations_file,
                       predictions_dir=predictions_dir,
                       dataset_type=dataset_type,
                       class_mapping_file=class_mapping_file,
                       output_metrics=output_metrics,
                       view_fiftyone=view_fiftyone,
                       case_sensitive=case_sensitive,
                       dataset_sample=dataset_subsample,
                       verbose=verbose
                       )

#### Setup dataset and add predictions

In [ ]:
evaluator.setup_dataset()
evaluator.add_predictions()

 100% |███████████████| 5023/5023 [5.1s elapsed, 0s remaining, 969.5 samples/s]       


#### Output results

In [ ]:
evaluator.evaluate_fiftyone()
if output_metrics is not None:
    metrics  = evaluator.generate_summary()
if view_fiftyone:
    evaluator.launch_fiftyone_session(port=view_fiftyone_port)

In [ ]:
metrics